In [0]:
from datetime import datetime
# reading data from partitoned table from the raw
staging_layer_df = spark.read.table("bike_details_raw").filter("partiton_date == '{}'".format(datetime.today().strftime('%Y-%m-%d')))

In [0]:
from pyspark.sql.functions import explode,col,current_timestamp
# Applying transformation explode 
staging_layer_df = staging_layer_df.select(explode('bikes').alias('data')).select(col('data.*')).withColumn('partiton_date', current_timestamp().cast("date"))

In [0]:
staging_layer_df.show(10,False)

+-----------+---------------------------------------------------------------------------------------------------------------+-----------+---------------------------------+-----------------------+-------+------------+-------------------------------------------------------------------------------+--------------------------------------+-----------------+-------------+------------+-------------+------+------+------------------+--------------------------+-------------------------------------------------------------------------------+--------------------------------------------+-----------------------------------+----+-------------+
|date_stolen|description                                                                                                    |external_id|frame_colors                     |frame_model            |id     |is_stock_img|large_img                                                                      |location_found                        |manufacturer_name|registry_nam

In [0]:
base_path = "/FileStore/tables/"
staging_layer_path = base_path + "staging_layer/"
# Writing data into staging layer with delta append mode 
from delta.tables import DeltaTable
from pyspark.sql import DataFrame, DataFrameWriter
staging_layer_df.write.format("delta").mode("append").partitionBy("partiton_date").save(staging_layer_path)

In [0]:
spark.sql(
    """
      DROP TABLE IF EXISTS {}
    """.format('bike_details_staging')
)

spark.sql(
  """
    CREATE TABLE {}
    USING DELTA 
    LOCATION "{}"
  """.format('bike_details_staging', staging_layer_path)
)

Out[6]: DataFrame[]

In [0]:
%sql
select * from bike_details_staging

date_stolen,description,external_id,frame_colors,frame_model,id,is_stock_img,large_img,location_found,manufacturer_name,registry_name,registry_url,serial,status,stolen,stolen_coordinates,stolen_location,thumb,title,url,year,partiton_date
1661363319,null,null,"List(Teal, Black)",MARLIN 5,1381170,false,null,null,Trek,null,null,WTU209JC0857S,stolen,true,"List(39.51, -84.75)","Oxford, OH 45056, US",null,2022 Trek MARLIN 5,https://bikeindex.org/bikes/1381170,2022,2022-08-24
1661362902,null,null,List(Green),RadRunner2,1381158,false,null,null,Rad Power Bikes,null,null,VR2J21V8772,stolen,true,null,US,null,2021 Rad Power Bikes RadRunner2,https://bikeindex.org/bikes/1381158,2021,2022-08-24
1661360400,null,null,List(Black),Flyer,1381160,false,null,null,Fairdale,null,null,16TDU0242,stolen,true,"List(39.76, -105.22)","Golden, CO 80401-1714, US",null,Fairdale Flyer,https://bikeindex.org/bikes/1381160,null,2022-08-24
1661360400,null,null,"List(Silver, gray or bare metal)",null,1381146,false,null,null,Trek,null,null,1191u0312,stolen,true,"List(44.06, -123.08)","Eugene, OR 97401, US",null,2022 Trek,https://bikeindex.org/bikes/1381146,2022,2022-08-24
1661356800,Its darker gray with red marking.and the letters for giant in darker gray. metal silver pedals and disk brakes.,null,"List(Silver, gray or bare metal, Red)",Mountain bike,1367548,false,null,null,Giant,null,null,C8AF0039,stolen,true,"List(53.55, -113.49)","Edmonton, CA",null,2020 Giant Mountain bike,https://bikeindex.org/bikes/1367548,2020,2022-08-24
1661363987,null,null,List(Black),Step Through,1381178,false,null,null,Lectric eBikes,null,null,210970564,stolen,true,"List(37.18, -113.5)","Washington, UT 84780, US",null,2021 Lectric eBikes Step Through,https://bikeindex.org/bikes/1381178,2021,2022-08-24
null,,null,List(Blue),LANA'I,1381061,false,null,"59 Silverstone Rd NW, Calgary, T3B, CA",Kona,null,null,Hidden,found,false,null,null,null,2020 Kona LANA'I,https://bikeindex.org/bikes/1381061,2020,2022-08-24
1661351400,"Has stickers on one side, also missing some parts and needs a key for the battery",null,List(White),RadRover 6 Step Through,1381188,false,https://files.bikeindex.org/uploads/Pu/626506/large_Rover6STPlus_white_side.png,null,Rad Power Bikes,null,null,RU6I21V5904,stolen,true,"List(34.17, -119.21)","Port Hueneme, CA 93043, US",https://files.bikeindex.org/uploads/Pu/626506/small_Rover6STPlus_white_side.png,2022 Rad Power Bikes RadRover 6 Step Through,https://bikeindex.org/bikes/1381188,2022,2022-08-24
1661351399,null,null,List(Purple),Custom Road,1381017,false,null,null,Mariposa,null,null,1431117,stolen,true,"List(44.49, -80.23)","Collingwood, L9Y 0E3, CA",null,2017 Mariposa Custom Road,https://bikeindex.org/bikes/1381017,2017,2022-08-24
1661349600,null,null,"List(Silver, gray or bare metal)",Verve 2 Disc,1381035,false,https://files.bikeindex.org/uploads/Pu/626453/large_verve2.png,null,Trek,null,null,WTU287XC0621S,stolen,true,"List(29.75, -95.36)","Houston, TX 77010, US",https://files.bikeindex.org/uploads/Pu/626453/small_verve2.png,2022 Trek Verve 2 Disc,https://bikeindex.org/bikes/1381035,2022,2022-08-24
